<a href="https://colab.research.google.com/github/hemanthkumar17/Machine-Learning/blob/main/Machine_Learning_Backprop_Calculations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

In [ ]:
import torch

In [ ]:
w1 = torch.tensor([
    [1, -1],
    [0, 0],
    [1, 1]
], dtype=torch.float32, requires_grad=True)
w2 = torch.tensor([[1, 0, -1]], dtype=torch.float32, requires_grad=True)
x = torch.tensor([
    [1, -2],
    [5, -9],
    [-3, 2]
], dtype=torch.float32).T
y = torch.tensor([
    [0.1],
    [1],
    [0.8]
], dtype=torch.float32)

In [ ]:
from torch import nn
class LazyModel(torch.nn.Module):
    def __init__(self, w1, w2):
        super(LazyModel, self).__init__()
        self.w1 = w1
        self.w2 = w2
        self.relu = nn.ReLU()
    def forward(self, x):
        self.out = torch.einsum("ij,jk->ik", self.w1, x)
        self.out_r = self.relu(self.out)
        self.out.retain_grad()
        self.out_r.retain_grad()
        self.out1 = torch.einsum("ij,jk->ik", self.w2, self.out_r)
        self.out1.retain_grad()
        self.out2 = torch.sigmoid(self.out1)
        self.out2.retain_grad()
        return self.out2

In [ ]:
model = LazyModel(w1, w2)

In [ ]:
lr = 1

In [ ]:
loss = nn.MSELoss()

pred = model(x)
print("*"*20 + "\tActivations\t" + "*"*20)
print(model.out)
print(model.out_r)
print(model.out1)
print(model.out2)
print("*"*20 + "\tPredictions\t" + "*"*20)
print(pred)
print(y)
l = loss(pred, y.T)
print(y.T)
print("*"*20 + "\tLoss and weights b4\t" + "*"*20)
print(l)

l.backward()

print(model.w1)
print(model.w2)
print("*"*20 + "\tGradients w1 and w2\t" + "*"*20)
print(model.w1.grad.data)
print(model.w2.grad.data)
print("*"*20 + "\tGradients o3, o2, a1 and o1\t" + "*"*20)
print(model.out2.grad.data)
print(model.out1.grad.data)
print(model.out_r.grad.data)
print(model.out.grad.data)
with torch.no_grad():
    model.w1.data -= lr * model.w1.grad.data
    model.w2.data -= lr * model.w2.grad.data

print("*"*20 + "\tWeights after update\t" + "*"*20)
print(model.w1)
print(model.w2)

********************	Activations	********************
tensor([[ 3., 14., -5.],
        [ 0.,  0.,  0.],
        [-1., -4., -1.]], grad_fn=<ViewBackward0>)
tensor([[ 3., 14.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.]], grad_fn=<ReluBackward0>)
tensor([[ 3., 14.,  0.]], grad_fn=<ViewBackward0>)
tensor([[0.9526, 1.0000, 0.5000]], grad_fn=<SigmoidBackward0>)
********************	Predictions	********************
tensor([[0.9526, 1.0000, 0.5000]], grad_fn=<SigmoidBackward0>)
tensor([[0.1000],
        [1.0000],
        [0.8000]])
tensor([[0.1000, 1.0000, 0.8000]])
********************	Loss and weights b4	********************
tensor(0.2723, grad_fn=<MseLossBackward0>)
tensor([[ 1., -1.],
        [ 0.,  0.],
        [ 1.,  1.]], requires_grad=True)
tensor([[ 1.,  0., -1.]], requires_grad=True)
********************	Gradients w1 and w2	********************
tensor([[ 0.0257, -0.0514],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000]])
tensor([[0.0770, 0.0000, 0.0000]])
*********

In [ ]:
model.out.grad

tensor([[ 2.5678e-02, -4.6422e-13,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00]])

In [ ]:
model.out1.grad

tensor([[ 2.5678e-02, -4.6422e-13, -5.0000e-02]])

In [ ]:
model.out2.grad

tensor([[ 5.6838e-01, -5.5631e-07, -2.0000e-01]])